In [4]:
import pandas as pd
import numpy as np
import json
import os
from tabulate import tabulate

In [30]:
cutoff = 16
potential = 'QHO'
folder_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\VQD\Destructive SWAP\VaryShots\{}".format(potential)
shots_list = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

In [31]:
converged_only = True
all_data = []

for shots in shots_list:

    median = []
    mean = []
    min = []
    e0 = []
    e1 = []
    e2 = []
    ratios = []

    data_path = folder_path + "\\{}\\{}_{}.json".format(shots,potential, cutoff)
    
    with open(data_path, 'r') as file:
        data = json.load(file)

    num_VQD = data['num_VQD']

    converged = [all(x) for x in data['success']]
    converged_indices = [i for i, success in enumerate(converged) if success]
    converged_runs = sum(converged)

    if converged_only:
        results = [data['results'][i] for i in converged_indices]
    else:
        results = data['results']

    success_i = [i for i, success in enumerate(data['success']) if all(success)]
    successful_results = [data['results'][i] for i in success_i]
    energies =np.sort(successful_results)

    run_times = data['run_times']#[data['run_times'][i] for i in converged_indices]
    time_series = np.array([pd.Timedelta(t).total_seconds() for t in run_times])
    mean_vqe_time = np.mean(time_series)
    time_taken = np.mean(mean_vqe_time)

    for e in energies:
        
        e0.append(e[0])
        e1.append(e[1])
        e2.append(e[2])
        r = abs((e[2] - e[1]) / (e[2] - e[0]))
        ratios.append(r)

    e0_exact = data['exact_eigenvalues'][0]
    e1_exact = data['exact_eigenvalues'][1]
    e2_exact = data['exact_eigenvalues'][2]
    exact_ratio = abs((e2_exact - e1_exact) / (e2_exact - e0_exact))

    row = {
    "shots": shots,
    'Converged Runs': f"{converged_runs}/100",
    'Mean VQD time (s)': f"{time_taken:.2f}",
    #"e0_exact": e0_exact,
    #"e1_exact": e1_exact,
    #"e2_exact": e2_exact,
    #"e0": np.median(e0),
    #"e1": np.median(e1),
    #"e2": np.median(e2),
    #"E01": e1_exact - e0_exact,
    #"E12": e2_exact - e1_exact,
    "exact_ratio": exact_ratio,
    "median_ratio": np.median(ratios),
    "delta_ratio": abs(exact_ratio-np.median(ratios))
    #"mean_ratio": np.mean(ratios),
    #"min_ratio": np.min(ratios),
    }

    all_data.append(row)

c:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\.venv\Lib\site-packages\numpy\_core\fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\.venv\Lib\site-packages\numpy\_core\_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [32]:
df = pd.DataFrame(all_data)
df['shots'] = df['shots'].astype(int)
df = df.sort_values('shots')

In [33]:
df

,shots,Converged Runs,Mean VQD time (s),exact_ratio,median_ratio,delta_ratio
0,1024,0/100,5739.72,4.440892e-16,NaN,NaN
2,2048,0/100,5691.67,4.440892e-16,NaN,NaN
4,4096,19/100,3637.87,4.440892e-16,0.051648,0.051648
5,8192,98/100,558.81,4.440892e-16,0.048639,0.048639
1,16384,100/100,171.91,4.440892e-16,0.026678,0.026678
3,32768,100/100,160.72,4.440892e-16,0.014696,0.014696


In [34]:
latex_table = tabulate(df.values, headers=df.columns, tablefmt="latex", floatfmt=".4f")
print(latex_table)

\begin{tabular}{rlrrrr}
\hline
   shots & Converged Runs   &   Mean VQD time (s) &   exact\_ratio &   median\_ratio &   delta\_ratio \\
\hline
    1024 & 0/100            &           5739.7200 &        0.0000 &       nan      &      nan      \\
    2048 & 0/100            &           5691.6700 &        0.0000 &       nan      &      nan      \\
    4096 & 19/100           &           3637.8700 &        0.0000 &         0.0516 &        0.0516 \\
    8192 & 98/100           &            558.8100 &        0.0000 &         0.0486 &        0.0486 \\
   16384 & 100/100          &            171.9100 &        0.0000 &         0.0267 &        0.0267 \\
   32768 & 100/100          &            160.7200 &        0.0000 &         0.0147 &        0.0147 \\
\hline
\end{tabular}
